In [ ]:
import cv2
import mediapipe as mp
import screen_brightness_control as sbc
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
import tkinter as tk
from tkinter import BooleanVar
import threading
from tkinter.ttk import Progressbar

# Inicialización de control de volumen
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
endpoint_volume = interface.QueryInterface(IAudioEndpointVolume)

# Función para setear el volumen principal del sistema
def set_volume(volume_level):
    volume_level = max(0, min(100, volume_level))
    endpoint_volume.SetMasterVolumeLevelScalar(volume_level / 100.0, None)

# Función para iniciar la detección de manos
def start_detection():
    global running
    running = True
    threading.Thread(target=hand_detection).start()

# Función para detener la detección de manos
def stop_detection():
    global running
    running = False

# Función principal de detección de manos
def hand_detection():
    global running, volume_label, brightness_label, volume_bar, brightness_bar, show_video_var, black_background_var
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)

    while running and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preparar fondo negro si es necesario
        black_frame = frame.copy()
        black_frame[:] = (0, 0, 0)

        # Procesar la imagen con Mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                mp_drawing = mp.solutions.drawing_utils
                label = handedness.classification[0].label

                # Dibujar puntos y conexiones en el fondo correspondiente
                target_frame = black_frame if black_background_var.get() else frame
                mp_drawing.draw_landmarks(
                    target_frame, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS, 
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

                # Contar dedos levantados
                finger_tips_ids = [4, 8, 12, 16, 20]
                finger_count_right = 0
                finger_count_left = 0

                for tip_id in finger_tips_ids:
                    if label == 'Right':  # Mano derecha
                        if tip_id == 4:  # Pulgar
                            if hand_landmarks.landmark[2].x < hand_landmarks.landmark[18].x:
                                if hand_landmarks.landmark[tip_id].x < hand_landmarks.landmark[tip_id - 1].x:
                                    finger_count_right += 1
                        else:  # Otros dedos
                            if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 2].y:
                                finger_count_right += 1
                    else:  # Mano izquierda
                        if tip_id == 4:  # Pulgar
                            if hand_landmarks.landmark[2].x > hand_landmarks.landmark[18].x:
                                if hand_landmarks.landmark[tip_id].x > hand_landmarks.landmark[tip_id - 1].x:
                                    finger_count_left += 1
                        else:
                            if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 2].y:
                                finger_count_left += 1

                # Calcular valores de volumen y brillo
                bright = 20 * finger_count_left
                vol = 20 * finger_count_right

                # Mostrar texto sobre el fondo correspondiente
                text = f'{label}: {finger_count_right} dedos levantados' if label == 'Right' else f'{label}: {finger_count_left} dedos levantados'
                position = (10, 30) if label == 'Right' else (10, 70)
                cv2.putText(target_frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                # Ajustar brillo y volumen
                if label == 'Right':
                    set_volume(vol)
                    volume_label.config(text=f"Volumen: {vol}%")
                    volume_bar['value'] = vol
                if label == 'Left':
                    sbc.set_brightness(bright)
                    brightness_label.config(text=f"Brillo: {bright}%")
                    brightness_bar['value'] = bright

        # Mostrar el video si está habilitado
        if show_video_var.get():
            cv2.imshow('Hand Detection', black_frame if black_background_var.get() else frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    hands.close()

# Configuración de la interfaz gráfica
app = tk.Tk()
app.title("Control de Volumen y Brillo")
app.geometry("400x400")

# Etiquetas y botones de control
volume_label = tk.Label(app, text="Volumen: 50%", font=("Helvetica", 12))
volume_label.pack(pady=10)

volume_bar = Progressbar(app, orient="horizontal", length=300, mode="determinate", maximum=100)
volume_bar.pack(pady=10)

brightness_label = tk.Label(app, text="Brillo: 50%", font=("Helvetica", 12))
brightness_label.pack(pady=10)

brightness_bar = Progressbar(app, orient="horizontal", length=300, mode="determinate", maximum=100)
brightness_bar.pack(pady=10)

start_button = tk.Button(app, text="Iniciar Detección", command=start_detection, bg="green", fg="white")
start_button.pack(pady=10)

stop_button = tk.Button(app, text="Detener Detección", command=stop_detection, bg="red", fg="white")
stop_button.pack(pady=10)

# Checkbox para mostrar/ocultar video
show_video_var = BooleanVar(value=True)
show_video_checkbox = tk.Checkbutton(app, text="Mostrar Video", variable=show_video_var)
show_video_checkbox.pack(pady=10)

# Checkbox para alternar entre fondo negro y fondo original
black_background_var = BooleanVar(value=False)
black_background_checkbox = tk.Checkbutton(app, text="Fondo Negro", variable=black_background_var)
black_background_checkbox.pack(pady=10)

# Variable para controlar el hilo de detección
running = False

# Ejecuta la interfaz
app.mainloop()